In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c sentiment-analysis-hse-fcl

sentiment-analysis-hse-fcl.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive')

zip_ref = zipfile.ZipFile("/content/sentiment-analysis-hse-fcl.zip", 'r')
zip_ref.extractall("/content/")
zip_ref.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch

In [ ]:
import json
from sklearn.model_selection import train_test_split
import bz2
import regex
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import re
import torch
import pandas as pd
import numpy as np

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
from tqdm import tqdm

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
train_texts = []
with open('x_train.txt', 'r', encoding='utf-8') as thefile:
    for row in tqdm(thefile):
        row = re.sub(r'[^\w\s]', '', row)
        row = row.lower()
        train_texts.append(row)

3600000it [00:38, 94645.79it/s]


In [ ]:
print(train_texts[354146])

a little much for a kids book has quite a few pretty tasteless racial and adulttype comments that is likely why it is no longer in print



In [ ]:
ratings = pd.read_csv('y_train.csv')['Probability'].values

In [ ]:
max_features = 20000
maxlen = 100
lemmatizer = WordNetLemmatizer()

In [ ]:
lem_texts = []
for text in tqdm(train_texts[:50000]):
    lem_texts.append(' '.join([lemmatizer.lemmatize(word) for word in text.split()]))

100%|██████████| 50000/50000 [00:12<00:00, 4008.29it/s]


In [ ]:
lem_texts[100]

'textbook book shipped quickly and wa in excellent condition a stated easy transaction would buy again'

In [ ]:
%%time
vectorizer = TfidfVectorizer(encoding='utf8', min_df=10)
_ = vectorizer.fit(lem_texts)

CPU times: user 2.44 s, sys: 48.8 ms, total: 2.49 s
Wall time: 2.48 s


In [ ]:
X = vectorizer.transform(lem_texts[:50000])
Y = (np.array(ratings[:50000])).astype(int)

In [ ]:
X, Y

(<50000x11719 sparse matrix of type '<class 'numpy.float64'>'
 	with 2577478 stored elements in Compressed Sparse Row format>,
 array([1, 1, 1, ..., 0, 1, 1]))

In [ ]:
X = X.tocoo()

In [ ]:
X

<50000x11719 sparse matrix of type '<class 'numpy.float64'>'
	with 2577478 stored elements in COOrdinate format>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7, shuffle=True)

In [ ]:
X_train

<15000x11719 sparse matrix of type '<class 'numpy.float64'>'
	with 776746 stored elements in Compressed Sparse Row format>

In [ ]:
X_test

<35000x11719 sparse matrix of type '<class 'numpy.float64'>'
	with 1800732 stored elements in Compressed Sparse Row format>

In [ ]:
X

<50000x11719 sparse matrix of type '<class 'numpy.float64'>'
	with 2577478 stored elements in COOrdinate format>

In [ ]:
def sp2tensor(X):
    values = X.data
    indices = np.vstack((X.row, X.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = X.shape

    X_t = torch.sparse.FloatTensor(i, v, torch.Size(shape))
    return X_t

In [ ]:
X_train_t, X_test_t = sp2tensor(X_train.tocoo()), sp2tensor(X_test.tocoo())

<ipython-input-23-2f9e3428c0e7>:9: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:621.)
  X_t = torch.sparse.FloatTensor(i, v, torch.Size(shape))


In [ ]:
y_train_t = torch.Tensor(y_train)
y_test_t = torch.Tensor(y_test)
X_train_dense = torch.tensor(X_train.toarray(), dtype=torch.float32).unsqueeze(1)
X_test_dense = torch.tensor(X_test.toarray(), dtype=torch.float32).unsqueeze(1)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer

class SentimentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(SentimentRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

def dense_collate_fn(batch):
    reviews, labels = zip(*batch)
    reviews = torch.stack(reviews)
    labels = torch.stack(labels)
    return reviews, labels

# гиперпараметры
input_size = X_train_dense.shape[2]
hidden_size = 256
output_size = 1
num_layers = 3
dropout = 0.3
num_epochs = 10
batch_size = 64
learning_rate = 0.001

model = SentimentRNN(input_size, hidden_size, output_size, num_layers)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

X_train_dense = torch.tensor(X_train.toarray(), dtype=torch.float32).unsqueeze(1)
X_test_dense = torch.tensor(X_test.toarray(), dtype=torch.float32).unsqueeze(1)
train_dataset = torch.utils.data.TensorDataset(X_train_dense, y_train_t)
test_dataset = torch.utils.data.TensorDataset(X_test_dense, y_test_t)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, collate_fn=dense_collate_fn)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, collate_fn=dense_collate_fn)

# обучение
model.train()
for epoch in range(num_epochs):
    for i, (reviews, labels) in enumerate(train_loader):
        outputs = model(reviews)
        loss = criterion(outputs.squeeze(), labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/10], Step [100/235], Loss: 0.3316
Epoch [1/10], Step [200/235], Loss: 0.3115
Epoch [2/10], Step [100/235], Loss: 0.1418
Epoch [2/10], Step [200/235], Loss: 0.1008
Epoch [3/10], Step [100/235], Loss: 0.1204
Epoch [3/10], Step [200/235], Loss: 0.1636
Epoch [4/10], Step [100/235], Loss: 0.0333
Epoch [4/10], Step [200/235], Loss: 0.2350
Epoch [5/10], Step [100/235], Loss: 0.0144
Epoch [5/10], Step [200/235], Loss: 0.0200
Epoch [6/10], Step [100/235], Loss: 0.0109
Epoch [6/10], Step [200/235], Loss: 0.0162
Epoch [7/10], Step [100/235], Loss: 0.0136
Epoch [7/10], Step [200/235], Loss: 0.0399
Epoch [8/10], Step [100/235], Loss: 0.0019
Epoch [8/10], Step [200/235], Loss: 0.0416
Epoch [9/10], Step [100/235], Loss: 0.0023
Epoch [9/10], Step [200/235], Loss: 0.0108
Epoch [10/10], Step [100/235], Loss: 0.0146
Epoch [10/10], Step [200/235], Loss: 0.0436


In [ ]:
""" model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for reviews, labels in test_loader:
        outputs = model(reviews)
        predicted = (torch.sigmoid(outputs.squeeze()) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test data: {100 * correct / total:.2f}%') """

" model.eval()\nwith torch.no_grad():\n    correct = 0\n    total = 0\n    for reviews, labels in test_loader:\n        outputs = model(reviews)\n        predicted = (torch.sigmoid(outputs.squeeze()) > 0.5).float()\n        total += labels.size(0)\n        correct += (predicted == labels).sum().item()\n\n    print(f'Accuracy of the model on the test data: {100 * correct / total:.2f}%') "

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from torch.utils.data import DataLoader, TensorDataset


test_texts = []
with open('x_test.txt', 'r', encoding='utf-8') as f:
    for row in f:
        row = re.sub(r'[^\w\s]', '', row)
        row = row.lower()
        test_texts.append(row)

X_fin_test = vectorizer.transform(test_texts)
X_fin_test_dense = torch.tensor(X_fin_test.toarray(), dtype=torch.float32).unsqueeze(1)

fin_test_dataset = TensorDataset(X_fin_test_dense)
fin_test_loader = DataLoader(dataset=fin_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def save_predictions_to_csv(predictions, file_path):
    ids = list(range(1, len(predictions) + 1))
    df = pd.DataFrame(list(zip(ids, predictions)), columns=['ID', 'Probability'])
    df.to_csv(file_path, index=False)

model.eval()
predictions = []
print('Saving...')
with torch.no_grad():
    for reviews in tqdm(fin_test_loader):
        outputs = model(reviews[0])
        predicted_probs = torch.sigmoid(outputs.squeeze()).numpy()
        predictions.extend(predicted_probs)

save_predictions_to_csv(predictions, 'submission.csv')

print("Predictions saved to submission.csv.")

Saving...


100%|██████████| 6250/6250 [00:19<00:00, 313.62it/s]


Predictions saved to submission.csv.
